In [ ]:
import librosa
import numpy as np
from pydub import AudioSegment
import os
from pydub.silence import detect_leading_silence

In [4]:
def strip_leading_and_trailing_silence(audio_segment, silence_thresh=-35.0, chunk_size=10):
    """
    Trims leading and trailing silence from an AudioSegment.
    """
    start_trim = detect_leading_silence(audio_segment, silence_thresh, chunk_size)
    end_trim = detect_leading_silence(audio_segment.reverse(), silence_thresh, chunk_size)
    duration = len(audio_segment)
    return audio_segment[start_trim:duration - end_trim]

In [ ]:


def f(input_path, min_onset_gap_seconds=0.05,
      silence_thresh_db=-32):
    """
    Splits .m4a audio files within a folder into segments based on detected onsets,
    enforcing a minimum gap between onsets, validating segment durations, and
    removing leading/trailing silence from each segment.

    Args:
        input_path (str): The path to the folder containing .m4a audio files.
        min_onset_gap_seconds (float): Minimum time in seconds required between
                                       detected onsets for them to be considered
                                       separate spikes/strums.
        silence_thresh_db (int): The loudness threshold in dBFS (decibels relative to
                                 full scale) below which audio is considered silence
                                 for trimming purposes. Default is -40 dBFS. Adjust
                                 this based on your background noise level.
        min_silence_len_ms (int): The minimum duration of silence (in milliseconds)
                                  to consider it actual silence for trimming.
                                  Default is 100 ms.
    """
    min_onset_gap_ms = min_onset_gap_seconds * 1000 # Convert to milliseconds for internal use

    for path1 in os.listdir(input_path):
        if path1.endswith(".m4a"):
            file_path = os.path.join(input_path, path1)

            try:
                y, sr = librosa.load(file_path, sr=None)
            except Exception as e:
                print(f"Error loading audio file {file_path} with librosa: {e}. Skipping.")
                continue

            # Detect onset frames with delta=0.5
            onset_frames_raw = librosa.onset.onset_detect(y=y, sr=sr, backtrack=True, delta=0.35)
            onset_times_raw = librosa.frames_to_time(onset_frames_raw, sr=sr)

            # --- Filter out closely spaced onsets to enforce minimum gap ---
            filtered_onset_times = []
            if len(onset_times_raw) > 0:
                filtered_onset_times.append(onset_times_raw[0]) # Always keep the first detected onset
                for i in range(1, len(onset_times_raw)):
                    # If the current onset is far enough from the last kept onset, add it
                    if (onset_times_raw[i] - filtered_onset_times[-1]) * 1000 >= min_onset_gap_ms:
                        filtered_onset_times.append(onset_times_raw[i])

            # Prepare onset_times for splitting.
            # Start the first segment from 0.0 seconds if there are any detected onsets.
            onset_times_for_splitting = [0.0] if filtered_onset_times else []
            if filtered_onset_times:
                onset_times_for_splitting.extend(filtered_onset_times)
            
            # Ensure the audio duration is added to capture the last segment
            audio_duration_seconds = librosa.get_duration(y=y, sr=sr)
            if not onset_times_for_splitting or onset_times_for_splitting[-1] < audio_duration_seconds - 0.01:
                 onset_times_for_splitting.append(audio_duration_seconds)
            
            # Remove duplicate times (can happen if first onset is very close to 0)
            onset_times_for_splitting = sorted(list(set(onset_times_for_splitting)))

            # Load audio using pydub for easier export
            try:
                audio = AudioSegment.from_file(file_path)
            except Exception as e:
                print(f"Error loading audio file with pydub {file_path}: {e}. Skipping.")
                continue

            # --- Define output folder name ---
            output_folder_base_name = path1.removesuffix('.m4a')
            if output_folder_base_name.startswith("l"):
                output_folder_base_name = output_folder_base_name.removeprefix("l")
            if output_folder_base_name.startswith("m"):
                output_folder_base_name = output_folder_base_name.removeprefix("m")
            output_folder_base_name = output_folder_base_name.removesuffix('_2').removesuffix('2')

            output_folder = os.path.join(input_path, output_folder_base_name)
            os.makedirs(output_folder, exist_ok=True)

            # Determine starting index for new files
            existing_wav_files = [f for f in os.listdir(output_folder) if f.startswith("strum_") and f.endswith(".wav")]
            max_num = 0
            for fname in existing_wav_files:
                try:
                    num_part = fname.replace("strum_", "").replace(".wav", "")
                    max_num = max(max_num, int(num_part))
                except ValueError:
                    pass
            start_index_for_new_files = max_num + 1

            # --- Export each strum segment with duration validation and silence removal ---
            segments_exported_count = 0
            for i in range(len(onset_times_for_splitting) - 1):
                start_ms = int(onset_times_for_splitting[i] * 1000)
                end_ms = int(onset_times_for_splitting[i + 1] * 1000)
                
                # Initial segment duration check (before any trimming)
                segment_duration_initial_ms = end_ms - start_ms

                if not (200 <= segment_duration_initial_ms <= 5000): # 1.5 seconds to 3 seconds
                    # print(f"Skipping segment (initial) from {start_ms/1000:.2f}s to {end_ms/1000:.2ff}s due to invalid duration ({segment_duration_initial_ms/1000:.2f}s).")
                    continue

                segment = audio[start_ms:end_ms]

                segment = strip_leading_and_trailing_silence(segment, silence_thresh_db)
                
                # After trimming, re-check duration to ensure it still meets criteria
                if not (200 <= len(segment) <= 5000):
                    # print(f"Skipping segment (after trim) from {start_ms/1000:.2f}s to {end_ms/1000:.2f}s due to invalid duration ({len(segment)/1000:.2f}s).")
                    continue

                # Export the segment
                segment_name = f"strum_{start_index_for_new_files + segments_exported_count}.wav"
                segment.export(os.path.join(output_folder, segment_name), format="wav")
                segments_exported_count += 1

            if segments_exported_count > 0:
                print(f"Split complete: {segments_exported_count} files saved in '{output_folder}' folder from '{file_path}'.")
            else:
                print(f"No valid segments found or exported for '{file_path}' after filtering and validation.")

In [7]:
f("./Farrel/ML DATASET GUITAR FINAL-20250520T082814Z-1-001/ML DATASET GUITAR FINAL/MINOR 7")

C:\Users\Bryan Santosa\AppData\Local\Temp\ipykernel_34280\1835319765.py:34: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=None)


Split complete: 14 files saved in './Farrel/ML DATASET GUITAR FINAL-20250520T082814Z-1-001/ML DATASET GUITAR FINAL/MINOR 7\A#min7' folder from './Farrel/ML DATASET GUITAR FINAL-20250520T082814Z-1-001/ML DATASET GUITAR FINAL/MINOR 7\lA#min7.m4a'.
Split complete: 15 files saved in './Farrel/ML DATASET GUITAR FINAL-20250520T082814Z-1-001/ML DATASET GUITAR FINAL/MINOR 7\A#min7' folder from './Farrel/ML DATASET GUITAR FINAL-20250520T082814Z-1-001/ML DATASET GUITAR FINAL/MINOR 7\lA#min7_2.m4a'.
Split complete: 13 files saved in './Farrel/ML DATASET GUITAR FINAL-20250520T082814Z-1-001/ML DATASET GUITAR FINAL/MINOR 7\Amin7' folder from './Farrel/ML DATASET GUITAR FINAL-20250520T082814Z-1-001/ML DATASET GUITAR FINAL/MINOR 7\lAmin7.m4a'.
Split complete: 12 files saved in './Farrel/ML DATASET GUITAR FINAL-20250520T082814Z-1-001/ML DATASET GUITAR FINAL/MINOR 7\Amin7' folder from './Farrel/ML DATASET GUITAR FINAL-20250520T082814Z-1-001/ML DATASET GUITAR FINAL/MINOR 7\lAmin7_2.m4a'.
Split complete: 